In [1]:
import gym
from gym import wrappers
env = gym.make("CartPole-v1")
"""
observation = env.reset()
for _ in range(100):
  env.render()
  action = env.action_space.sample() # your agent here (this takes random actions)
  observation, reward, done, info = env.step(action)

  if done:
        break
    #observation = env.reset()
env.close()
"""

'\nobservation = env.reset()\nfor _ in range(100):\n  env.render()\n  action = env.action_space.sample() # your agent here (this takes random actions)\n  observation, reward, done, info = env.step(action)\n\n  if done:\n        break\n    #observation = env.reset()\nenv.close()\n'

In [2]:
from ppo import PPO
from config_utils import get_config
from tqdm.notebook import tqdm

In [3]:
config = get_config("configs/config-2020_06_02.json")
# config = get_config("configs/config-default_mujoco.json")
print(config)

# num_epochs = 1200
num_epochs = 1
minibatch_size = 1

if config.has_training:
    num_epochs = config.training.get("num_epochs", num_epochs)
    minibatch_size = config.training.get("minibatch_size", minibatch_size)

{'ppo_hyperparameters': {'epsilon': 0.2, 'discount_factor': 0.9, 'actor_critic': {'actor': {'learning_rate': 0.005}, 'critic': {'learning_rate': 0.005, 'alpha': 0.75}}}}


In [4]:
# ppo = PPO(env, 4, env.action_space.n, 0.2, 0.90, 0.75)
ppo = PPO(env, config)

In [5]:
for _ in tqdm(range(num_epochs), desc="epoch"):
    for _ in tqdm(range(minibatch_size), desc="minibatch"):
        ppo.step()

In [6]:
import torch
from torch.autograd import Variable

In [7]:
#env_new = wrappers.Monitor(env, './video/', force = True)
times = 0
env_new = env
observation = env_new.reset()
observation = Variable(torch.from_numpy(observation), requires_grad=False).detach()
for _ in range(100):
# for _ in range(1):
    env_new.render()
    action = torch.argmax(ppo.pi(observation.float())) 
    observation, reward, done, info = env_new.step(action.detach().numpy())
    observation = Variable(torch.from_numpy(observation), requires_grad=False).detach()
    if not done:
        times += 1
        
print(times)
env_new.close()

19
